## Practical 1: word2vec
<p>Oxford CS - Deep NLP 2017<br>
https://www.cs.ox.ac.uk/teaching/courses/2016-2017/dl/</p>
<p>[Yannis Assael, Brendan Shillingford, Chris Dyer]</p>

This practical is presented as an IPython Notebook, with the code written for recent versions of **Python 3**. The code in this practical will not work with Python 2 unless you modify it. If you are using your own Python installation, ensure you have a setup identical to that described in the installation shell script (which is intended for use with the department lab machines). We will be unable to support installation on personal machines due to time constraints, so please use the lab machines and the setup script if you are unfamiliar with how to install Anaconda.

To execute a notebook cell, press `shift-enter`. The return value of the last command will be displayed, if it is not `None`.

Potentially useful library documentation, references, and resources:

* IPython notebooks: <https://ipython.org/ipython-doc/3/notebook/notebook.html#introduction>
* Numpy numerical array library: <https://docs.scipy.org/doc/>
* Gensim's word2vec: <https://radimrehurek.com/gensim/models/word2vec.html>
* Bokeh interactive plots: <http://bokeh.pydata.org/en/latest/> (we provide plotting code here, but click the thumbnails for more examples to copy-paste)
* scikit-learn ML library (aka `sklearn`): <http://scikit-learn.org/stable/documentation.html>
* nltk NLP toolkit: <http://www.nltk.org/>
* tutorial for processing xml in python using `lxml`: <http://lxml.de/tutorial.html> (we did this for you below, but in case you need it in the future)

In [1]:
import numpy as np
import os
from random import shuffle
import re

In [2]:
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

### Part 0: Download the TED dataset

In [3]:
import urllib.request
import zipfile
import lxml.etree

In [4]:
# Download the dataset if it's not already there: this may take a minute as it is 75MB
if not os.path.isfile('ted_en-20160408.zip'):
    urllib.request.urlretrieve("https://wit3.fbk.eu/get.php?path=XML_releases/xml/ted_en-20160408.zip&filename=ted_en-20160408.zip", filename="ted_en-20160408.zip")

In [5]:
# For now, we're only interested in the subtitle text, so let's extract that from the XML:
with zipfile.ZipFile('ted_en-20160408.zip', 'r') as z:
    doc = lxml.etree.parse(z.open('ted_en-20160408.xml', 'r'))
input_text = '\n'.join(doc.xpath('//content/text()'))
del doc

### Part 1: Preprocessing

In this part, we attempt to clean up the raw subtitles a bit, so that we get only sentences. The following substring shows examples of what we're trying to get rid of. Since it's hard to define precisely what we want to get rid of, we'll just use some simple heuristics.

In [6]:
i = input_text.find("Hyowon Gweon: See this?")
input_text[i-20:i+150]

' baby does.\n(Video) Hyowon Gweon: See this? (Ball squeaks) Did you see that? (Ball squeaks) Cool. See this one? (Ball squeaks) Wow.\nLaura Schulz: Told you. (Laughs)\n(Vide'

Let's start by removing all parenthesized strings using a regex:

In [7]:
input_text_noparens = re.sub(r'\([^)]*\)', '', input_text)

We can verify the same location in the text is now clean as follows. We won't worry about the irregular spaces since we'll later split the text into sentences and tokenize it anyway.

In [8]:
i = input_text_noparens.find("Hyowon Gweon: See this?")
input_text_noparens[i-20:i+150]

"hat the baby does.\n Hyowon Gweon: See this?  Did you see that?  Cool. See this one?  Wow.\nLaura Schulz: Told you. \n HG: See this one?  Hey Clara, this one's for you. You "

Now, let's attempt to remove speakers' names that occur at the beginning of a line, by deleting pieces of the form "`<up to 20 characters>:`", as shown in this example. Of course, this is an imperfect heuristic. 

In [9]:
sentences_strings_ted = []
for line in input_text_noparens.split('\n'):
    m = re.match(r'^(?:(?P<precolon>[^:]{,20}):)?(?P<postcolon>.*)$', line)
    sentences_strings_ted.extend(sent for sent in m.groupdict()['postcolon'].split('.') if sent)

# Uncomment if you need to save some RAM: these strings are about 50MB.
# del input_text, input_text_noparens

# Let's view the first few:
sentences_strings_ted[:5]

["Here are two reasons companies fail: they only do more of the same, or they only do what's new",
 'To me the real, real solution to quality growth is figuring out the balance between two activities: exploration and exploitation',
 ' Both are necessary, but it can be too much of a good thing',
 'Consider Facit',
 " I'm actually old enough to remember them"]

Now that we have sentences, we're ready to tokenize each of them into words. This tokenization is imperfect, of course. For instance, how many tokens is "can't", and where/how do we split it? We'll take the simplest naive approach of splitting on spaces. Before splitting, we remove non-alphanumeric characters, such as punctuation. You may want to consider the following question: why do we replace these characters with spaces rather than deleting them? Think of a case where this yields a different answer.

In [10]:
sentences_ted = []
for sent_str in sentences_strings_ted:
    tokens = re.sub(r"[^a-z0-9]+", " ", sent_str.lower()).split()
    sentences_ted.append(tokens)

Two sample processed sentences:

In [11]:
len(sentences_ted)

266694

In [12]:
print(sentences_ted[0])
print(sentences_ted[1])

['here', 'are', 'two', 'reasons', 'companies', 'fail', 'they', 'only', 'do', 'more', 'of', 'the', 'same', 'or', 'they', 'only', 'do', 'what', 's', 'new']
['to', 'me', 'the', 'real', 'real', 'solution', 'to', 'quality', 'growth', 'is', 'figuring', 'out', 'the', 'balance', 'between', 'two', 'activities', 'exploration', 'and', 'exploitation']


### Part 2: Word Frequencies

If you store the counts of the top 1000 words in a list called `counts_ted_top1000`, the code below will plot the histogram requested in the writeup.

In [13]:
word_dict = {}
for sentence_str in sentences_ted:
    for word in sentence_str:
        if word in word_dict:
            word_dict[word]+=1
        else:
            word_dict[word]=1

In [14]:
len(word_dict)

54454

In [15]:
counts_ted_top1000_tuple = sorted(word_dict.items(), key=lambda x:-x[1])
counts_ted_top1000 = list(map(lambda x: int(x[1]),counts_ted_top1000_tuple))[0:1000]

In [57]:
words_top_ted = list(map(lambda x: x[0],counts_ted_top1000_tuple))[0:1000]

Plot distribution of top-1000 words

In [16]:
hist, edges = np.histogram(counts_ted_top1000, density=True, bins=100, normed=True)

p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="Top-1000 words distribution")
p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], line_color="#555555")
show(p)

### Part 3: Train Word2Vec

In [18]:
from gensim.models import Word2Vec

In [19]:
model_ted = Word2Vec(sentences_ted, min_count=10)

In [36]:
len(model_ted.wv.vocab)

14427

### Part 4: Ted Learnt Representations

Finding similar words: (see gensim docs for more functionality of `most_similar`)

In [46]:
model_ted.wv.most_similar("man")

[('woman', 0.8457483649253845),
 ('guy', 0.8256943225860596),
 ('lady', 0.7865980863571167),
 ('gentleman', 0.7724959850311279),
 ('soldier', 0.7602518796920776),
 ('boy', 0.7510353326797485),
 ('girl', 0.7372521162033081),
 ('kid', 0.7025079727172852),
 ('surgeon', 0.6831142902374268),
 ('rabbi', 0.676139235496521)]

In [47]:
model_ted.wv.most_similar("computer")

[('machine', 0.7295936346054077),
 ('software', 0.7248227000236511),
 ('robot', 0.692204475402832),
 ('program', 0.6568593978881836),
 ('device', 0.643268883228302),
 ('3d', 0.6412458419799805),
 ('interface', 0.6358497142791748),
 ('chip', 0.6304893493652344),
 ('mechanical', 0.6278794407844543),
 ('video', 0.6180953979492188)]

In [49]:
model_ted.wv.most_similar(positive=['man', 'computer'], topn=6)

[('guy', 0.6942595839500427),
 ('kid', 0.6854121685028076),
 ('film', 0.6846330165863037),
 ('piano', 0.6833221912384033),
 ('surgeon', 0.6784018278121948),
 ('lady', 0.6699389815330505)]

In [48]:
model_ted.wv.most_similar(positive=['fuck','girl'], topn=11)

[('amy', 0.8000012636184692),
 ('emily', 0.7871090769767761),
 ('boy', 0.7644047141075134),
 ('pianist', 0.7494537830352783),
 ('robinson', 0.7484829425811768),
 ('bride', 0.74388587474823),
 ('dancer', 0.736926794052124),
 ('princess', 0.7311553955078125),
 ('mary', 0.7305142879486084),
 ('jones', 0.7238016128540039),
 ('dickinson', 0.7232913970947266)]

In [50]:
model_ted.wv.most_similar(positive=['school','doctor'], topn=11)

[('teacher', 0.7765592932701111),
 ('college', 0.7705076932907104),
 ('student', 0.739568829536438),
 ('hospital', 0.729615330696106),
 ('clinic', 0.6952590942382812),
 ('mom', 0.6917791366577148),
 ('friend', 0.6883143186569214),
 ('dad', 0.6882261037826538),
 ('office', 0.6880638599395752),
 ('husband', 0.6878767013549805),
 ('kid', 0.6866960525512695)]

In [51]:
model_ted.wv.most_similar(positive=['trump','ass'], topn=11)

[('sweetheart', 0.8207674026489258),
 ('thanking', 0.8143536448478699),
 ('whispering', 0.8123565316200256),
 ('escaping', 0.8089722394943237),
 ('anthony', 0.8050616383552551),
 ('successor', 0.803871750831604),
 ('gabriel', 0.8005318641662598),
 ('pupils', 0.7905108332633972),
 ('reassure', 0.7881026268005371),
 ('violate', 0.7876425385475159),
 ('sermons', 0.7864512205123901)]

In [61]:
model_ted.wv.most_similar(positive=['spent'], topn=11)

[('spend', 0.6648608446121216),
 ('lived', 0.5705054402351379),
 ('lost', 0.5593311190605164),
 ('worked', 0.54289710521698),
 ('experienced', 0.5374667644500732),
 ('started', 0.530876100063324),
 ('gained', 0.5300005674362183),
 ('received', 0.5267026424407959),
 ('met', 0.5264135599136353),
 ('reached', 0.5210707187652588),
 ('watched', 0.517153799533844)]

In [62]:
model_ted.wv.most_similar(positive=['30'], topn=11)

[('20', 0.9395052194595337),
 ('40', 0.9344936013221741),
 ('50', 0.923672080039978),
 ('25', 0.9102098345756531),
 ('60', 0.9044216871261597),
 ('15', 0.9041323065757751),
 ('10', 0.900549590587616),
 ('45', 0.8952577710151672),
 ('35', 0.8781905174255371),
 ('200', 0.8705756664276123),
 ('ten', 0.8642096519470215)]

#### t-SNE visualization
To use the t-SNE code below, first put a list of the top 1000 words (as strings) into a variable `words_top_ted`. The following code gets the corresponding vectors from the model, assuming it's called `model_ted`:

In [58]:
# This assumes words_top_ted is a list of strings, the top 1000 words
words_top_vec_ted = model_ted[words_top_ted]

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [59]:
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2, random_state=0)
words_top_ted_tsne = tsne.fit_transform(words_top_vec_ted)

In [60]:
p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE for most common words")

source = ColumnDataSource(data=dict(x1=words_top_ted_tsne[:,0],
                                    x2=words_top_ted_tsne[:,1],
                                    names=words_top_ted))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)

### Part 5: Wiki Learnt Representations

Download dataset

In [63]:
if not os.path.isfile('wikitext-103-raw-v1.zip'):
    urllib.request.urlretrieve("https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-103-raw-v1.zip", filename="wikitext-103-raw-v1.zip")

In [64]:
with zipfile.ZipFile('wikitext-103-raw-v1.zip', 'r') as z:
    input_text = str(z.open('wikitext-103-raw/wiki.train.raw', 'r').read(), encoding='utf-8') # Thanks Robert Bastian

Preprocess sentences (note that it's important to remove small sentences for performance)

In [65]:
sentences_wiki = []
for line in input_text.split('\n'):
    s = [x for x in line.split('.') if x and len(x.split()) >= 5]
    sentences_wiki.extend(s)
    
for s_i in range(len(sentences_wiki)):
    sentences_wiki[s_i] = re.sub("[^a-z]", " ", sentences_wiki[s_i].lower())
    sentences_wiki[s_i] = re.sub(r'\([^)]*\)', '', sentences_wiki[s_i])
del input_text

In [66]:
# sample 1/5 of the data
shuffle(sentences_wiki)
print(len(sentences_wiki))
sentences_wiki = sentences_wiki[:int(len(sentences_wiki)/5)]
print(len(sentences_wiki))

4267112
853422


In [68]:
sentences_wiki[1]

' his remains were reinterred in the monument to the independence of brazil   along with those of maria leopoldina and am lie   in the city of s o paulo '

Now, repeat all the same steps that you performed above. You should be able to reuse essentially all the code.

In [70]:
sentences_wiki_token = []
for sent_str in sentences_wiki:
    tokens = re.sub(r"[^a-z0-9]+", " ", sent_str.lower()).split()
    sentences_wiki_token.append(tokens)

In [71]:
sentences_wiki_token[0]

['the',
 'muffled',
 'explosion',
 'limited',
 'the',
 'damage',
 'to',
 'her',
 'hull']

In [73]:
word_dict = {}
for sentence_str in sentences_wiki_token:
    for word in sentence_str:
        if word in word_dict:
            word_dict[word]+=1
        else:
            word_dict[word]=1

In [74]:
len(word_dict)

235048

In [76]:
counts_wiki_top1000_tuple = sorted(word_dict.items(), key=lambda x:-x[1])
counts_wiki_top1000 = list(map(lambda x: int(x[1]),counts_wiki_top1000_tuple))[0:1000]
words_top_wiki = list(map(lambda x: x[0],counts_wiki_top1000_tuple))[0:1000]

In [77]:
counts_wiki_top1000_tuple

[('the', 1285954),
 ('of', 549429),
 ('and', 499035),
 ('in', 433729),
 ('to', 398382),
 ('a', 358985),
 ('was', 214932),
 ('s', 163699),
 ('on', 153694),
 ('as', 144715),
 ('for', 139804),
 ('that', 138164),
 ('with', 130297),
 ('by', 126070),
 ('is', 103030),
 ('his', 93449),
 ('at', 92000),
 ('he', 91917),
 ('from', 89273),
 ('it', 88919),
 ('were', 71136),
 ('an', 64526),
 ('had', 58591),
 ('which', 54959),
 ('be', 47572),
 ('this', 43109),
 ('but', 42438),
 ('are', 40852),
 ('first', 40673),
 ('not', 39557),
 ('their', 38682),
 ('after', 38247),
 ('one', 38010),
 ('her', 37473),
 ('also', 36639),
 ('its', 36514),
 ('they', 35269),
 ('two', 34094),
 ('or', 33592),
 ('have', 32794),
 ('who', 32012),
 ('been', 30318),
 ('new', 30086),
 ('has', 29999),
 ('she', 29195),
 ('when', 27256),
 ('during', 26424),
 ('time', 26233),
 ('other', 25877),
 ('would', 25394),
 ('into', 24655),
 ('all', 24114),
 ('i', 23815),
 ('more', 22912),
 ('over', 21477),
 ('while', 21292),
 ('game', 19867),
 (

In [81]:
#model
model_wiki = Word2Vec(sentences_wiki_token, min_count=10)
len(model_wiki.wv.vocab)

48594

In [94]:
model_wiki.wv.most_similar(positive=['erotic'], topn=11)

[('melodrama', 0.7754218578338623),
 ('irony', 0.7617379426956177),
 ('witty', 0.7516524195671082),
 ('seductive', 0.7494978904724121),
 ('humorous', 0.7465130090713501),
 ('provocative', 0.7452986836433411),
 ('inspirational', 0.7365840673446655),
 ('understated', 0.7348012924194336),
 ('satirical', 0.7342373132705688),
 ('poignant', 0.7336341142654419),
 ('surreal', 0.731939435005188)]

#### t-SNE visualization

In [82]:
# This assumes words_top_wiki is a list of strings, the top 1000 words
words_top_vec_wiki = model_wiki[words_top_wiki]

tsne = TSNE(n_components=2, random_state=0)
words_top_wiki_tsne = tsne.fit_transform(words_top_vec_wiki)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [83]:
p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE for most common words")

source = ColumnDataSource(data=dict(x1=words_top_wiki_tsne[:,0],
                                    x2=words_top_wiki_tsne[:,1],
                                    names=words_top_wiki))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)